# Article for Journal of Open Source Software
Martin Vonk & Aaron Peche (2026)

This notebook replicates the results presented in the article submitted to the Journal of Open Source Software (JOSS). The article can be found here: ... doi.org/...

JOSS is a developer-friendly, open-access academic journal (ISSN 2475-9066) dedicated to research software packages and features a formal peer-review process. The pre-review and review of the Pedon package are publicly available in issues ... and ..., respectively.

## Setup

In [ ]:
import matplotlib as mpl
import numpy as np
from cycler import cycler

import pedon as pe

rc_params = {
    "axes.prop_cycle": cycler(
        color=[
            "#3f90da",
            "#ffa90e",
            "#bd1f01",
            "#94a4a2",
            "#832db6",
            "#a96b59",
            "#e76300",
            "#b9ac70",
            "#717581",
            "#92dadd",
        ]
    ),
    "axes.titlesize": 10.0,
    "axes.grid": True,
    "axes.labelsize": 9.0,
    "xtick.labelsize": 8.0,
    "ytick.labelsize": 8.0,
    "legend.fontsize": 8.0,
    "legend.framealpha": 1.0,
    "figure.figsize": [7.0, 6.0],
}

mpl.rcParams.update(rc_params)

## Defining a soil model

In [ ]:
mg = pe.Genuchten(
    k_s=106.1,  # saturated conductivity (cm/d)
    theta_r=0.065,  # residual water content (-)
    theta_s=0.41,  # saturated water content (-)
    alpha=0.075,  # shape parameter (1/cm)
    n=1.89,  # shape parameter (-)
)

h = np.logspace(-2, 6, 9)  # pressure head (cm)
theta = mg.theta(h)  # water content (-) at pressure head values
k = mg.k(h)  # hydraulic conductivity (cm/d) at pressure head values

np.array(list(zip(h, theta, k)))

## Loading soil models from dataset

In [ ]:
hydrus = pe.Soil("Sand").from_name(pe.Genuchten, source="HYDRUS")
vs2d = pe.Soil("Sand").from_name(pe.Brooks, source="VS2D")
staring = pe.Soil("B01").from_name(pe.Genuchten, source="Staring_2018")

ax = hydrus.model.plot()
vs2d.model.plot(ax=ax)
staring.model.plot(ax=ax)

## Obtaining a soil model from pedotransfer function and databases 

In [ ]:
sand_p = 40.0  # sand (%)
clay_p = 10.0  # clay (%)
cosby: pe.Brooks = pe.SoilSample(sand_p=sand_p, clay_p=clay_p).cosby()

ks = 1e-2  # saturated hydraulic conductivity (m/d)
hypags: pe.Genuchten = pe.SoilSample(k=ks).hypags()

## Fitting a soil model

In [ ]:
bc = pe.SoilSample(h=h, theta=theta, k=k).fit(pe.Brooks)

ax = mg.plot()
bc.plot(ax=ax)
ax.set_ylim(h[0], h[-1])
fig = ax.get_figure()
fig.set_figheight(4.0)
fig.set_figwidth(2.5)

# ax.legend(["Mualem-van Genuchten", "Brooks-Corey"], loc="lower right", bbox_to_anchor=(1.0, 1.0), frameon=False)
ax.legend(["Mualem-van Genuchten", "Brooks-Corey"], loc="upper right")
ax.set_ylabel("|\N{GREEK SMALL LETTER PSI}| (cm)")
ax.set_xlabel("\N{GREEK SMALL LETTER THETA} (-)")

fig.savefig("figures/swrc_fit.png", dpi=300, bbox_inches="tight")